# Supply Data Analysis

## Business Objectives


Important part of our business is a supply/demand balance. We can’t control demand but we can shift some supply to necessary hours to cover more demand during peaks.

As part of the task you will have sample supply and demand data over a few weeks in a single city a few weeks after launch.

#### We need to understand:
- What is the supply to demand dynamic and whether they match?
- Where are the hours of oversupply? Can we shift some of them to undersupply hours?

## Data understanding

#### Hourly driver activity:
- Date – date + hour for which the row of data is presented
- Active drivers – number of active drivers (any level of activity) available during time period
- Online (h) – total supply hours that were available during time period
- Has booking (h) – total hours during which drivers had a client booking (any state)
- Waiting for booking (h) – total hours which drivers spent waiting for booking
- Busy (h) – total hours which drivers were not available to take orders in
- Hours per active driver – aka HPA - average number of hours each driver was online during time period
- Rides per online hour – aka RPH – avg. finished trips per online hour during period
- Finished Rides – number of finished trips during period

#### Hourly Overview Search:
- Date – date + hour for which the row of data is presented
- People saw 0 cars (unique) – number of users who didn’t not see a car.
- People saw +1 cars (unique) – number of users who saw a car.
- Coverage Ratio (unique) – % of users who saw the car.

## Import Required Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.express as px

## Data Sourcing Cleaning and Preparation

### Supply data info: Hourly Driver Activity

In [ ]:
driver_activity = pd.read_csv('Hourly_DriverActivity_1.csv')
driver_activity.head()

In [ ]:
# checking the shape i.e row and column
driver_activity.shape

In [ ]:
overview_search = pd.read_csv('Hourly_OverviewSearch_1.csv')
overview_search.head()

In [ ]:
# checking the shape i.e row and column
overview_search.shape

In [ ]:
# Joining Dataframes
drivers_and_users_merge = pd.merge(driver_activity, overview_search, how='left', left_on = 'Date', right_on = 'Date')
drivers_and_users_merge.head()


In [ ]:
#Rename columns
drivers_and_users = drivers_and_users_merge.rename(columns={"Active drivers":"Demand", 
                                  "Online (h)": "Online", 
                                  "Has booking (h)": "Has_booking", 
                                  "Waiting for booking (h)": "Waiting_for_booking", 
                                  "Busy (h)": "Busy",
                                  "Hours per active driver": "HPA",
                                  "Rides per online hour": "RPH",
                                  "Finished Rides": "Finished_Rides",
                                  "Coverage Ratio (unique)": "Coverage_Ratio",
                                  
                                  "People saw 0 cars (unique)": "Unavailable", 
                                  "People saw +1 cars (unique)": "Available"})
drivers_and_users.head()

In [ ]:
drivers_and_users['Date'] = pd.to_datetime(drivers_and_users['Date'])
drivers_and_users['Dates'] = drivers_and_users['Date'].dt.strftime('%Y-%m-%d')
drivers_and_users['Time'] = drivers_and_users['Date'].dt.strftime('%H')
drivers_and_users['Week_Number'] = drivers_and_users['Date'].dt.week
drivers_and_users['Week_Name'] = drivers_and_users['Date'].dt.day_name()

In [ ]:
#checking for null values column wise
drivers_and_users.isnull().sum()

In [ ]:
#Checking null value percetage in each column
round(100*(drivers_and_users.isnull().sum()/len(drivers_and_users.index)), 2)

In [ ]:
drivers_and_users.describe().round(1)

In [ ]:
drivers_and_users.head()

In [ ]:
# Group by day of week and time
columns = {'Demand':'mean', 'Online':'mean', 
           'Unavailable':'mean', 'Available':'mean', 
           'Has_booking': 'mean', 'Waiting_for_booking':'mean', 'Busy':'mean','Finished_Rides':'mean'}
groupby_time_drivers_and_users = drivers_and_users.groupby(['Week_Name','Time'], as_index = False).agg(columns)
groupby_time_drivers_and_users['Week_Name'] = pd.Categorical(groupby_time_drivers_and_users['Week_Name'], categories=
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'],
    ordered=True)
groupby_time_drivers_and_users['RPH_by_time'] = (groupby_time_drivers_and_users.Finished_Rides/groupby_time_drivers_and_users.Online).round(2)
groupby_time_drivers_and_users['HRA_by_time'] = (groupby_time_drivers_and_users.Online/groupby_time_drivers_and_users.Demand).round(2)
groupby_time_drivers_and_users['Conversion'] = (groupby_time_drivers_and_users.Finished_Rides/groupby_time_drivers_and_users.Available).round(2)
groupby_time_drivers_and_users['Coverage_Ratio_by_time'] = (groupby_time_drivers_and_users.Available/(groupby_time_drivers_and_users.Available+groupby_time_drivers_and_users.Unavailable)).round(2)

groupby_time_drivers_and_users.head()

## Data Analysis

### 1. Show which 36 hours in a week are most undersupplied. Show/describe your decision based on sample data.

In [ ]:
groupby_time_drivers_and_users['Supply'] = (groupby_time_drivers_and_users.Unavailable + 
                                            groupby_time_drivers_and_users.Available)
groupby_time_drivers_and_users['SD_difference'] = (groupby_time_drivers_and_users.Supply - 
                                                   groupby_time_drivers_and_users.Demand)
groupby_time_drivers_and_users.head()

In [ ]:
# 36 hours in a week are most undersupplied by Unavailable
hours_36 = groupby_time_drivers_and_users.sort_values(by=['Unavailable'], ascending=False)
hours_36.head(37)

### 2. 24-hour curve of average supply and demand (to illustrate match/mismatch).

In [ ]:
columns = {'Demand':'mean', 'Supply':'mean'}
groupby_day_drivers_and_users = (groupby_time_drivers_and_users.
                                 groupby(['Time'], as_index = False).
                                 agg(columns).round(1))
groupby_day_drivers_and_users

In [ ]:
# Illustration match/mismatch supply and demand
plt.plot(groupby_day_drivers_and_users['Time'], 
         groupby_day_drivers_and_users['Demand'], 
         color = 'orange', label = 'Demand');
plt.plot(groupby_day_drivers_and_users['Time'], 
         groupby_day_drivers_and_users['Supply'], 
         color = 'blue', label = 'Supply');
plt.xlabel('Time');
plt.ylabel('Quantity');
plt.title('24-hour curve of average supply and demand')
plt.legend();

There is a big lack of drivers from 5-6 am, in the afternoon from 10-17 and after 20 o'clock.

### 3. Visualisation of hours where we lack supply during a weekly period. This one we can send to drivers to show when to online for extra hours.

In [ ]:
groupby_time_drivers_and_users['Week_name_time'] = (groupby_time_drivers_and_users.Week_Name.astype(str) + " " + 
                                                    groupby_time_drivers_and_users.Time)
groupby_time_drivers_and_users

In [ ]:
columns = {'SD_difference':'mean'}
groupby_day_drivers_and_users = (groupby_time_drivers_and_users.
                                 groupby(['Week_name_time'], as_index = False).
                                 agg(columns).round(1))

lack_supply = groupby_day_drivers_and_users[groupby_day_drivers_and_users["SD_difference"] < 0]
lack_supply.head()

In [ ]:
sns.set(rc={'figure.figsize':(30,5)})
ax = sns.barplot(x="Week_name_time", y="SD_difference", data=lack_supply)
ax.tick_params(axis='x', rotation=90)
ax.set_xlabel("Day of the week by time", fontsize = 12)
ax.set_ylabel("Number of unavailable cars", fontsize = 12)
ax.set_title('Lack supply during', fontsize = 16)


### 4. Estimate number of hours needed to ensure we have a high Coverage Ratio during most peak hours.

In [ ]:
groupby_time_drivers_and_users.head()

In [ ]:
sns.set(rc={'figure.figsize':(20,5)})
ax = sns.boxplot(x="Week_Name", y="Coverage_Ratio_by_time", data=groupby_time_drivers_and_users)
ax.tick_params(axis='x', rotation=90)


In [ ]:
ax = sns.boxplot(x="Week_Number", y="Demand", data=drivers_and_users)

In [ ]:
ax = sns.boxplot(x="Time", y="Demand", data=drivers_and_users)

In [ ]:
ax = sns.boxplot(x="Time", y="Unavailable", data=drivers_and_users)

In [ ]:
groupby_time_drivers_and_users['Unavailable'].hist()

In [ ]:
groupby_time_drivers_and_users['Finished_Rides'].hist()

In [ ]:
sns.heatmap(groupby_time_drivers_and_users.corr())

### 5. Calculate levels of guaranteed hourly earnings we can offer to drivers during 36 weekly hours with highest demand without losing money + how much extra hours we want to get to capture missed demand.
- Assume that Finished Rides have an average value of €10 (80% goes to driver, 20% is our revenue).
- Assume the same level of demand with increased supply, base it on RPH over 3 hour periods, but with increased supply.
- Assume that with extra hours we will capture “missed coverage” or people attributed to “People saw 0 cars” in demand data.


In [ ]:
hours_36.head(37)

In [ ]:
hours_36.describe().round(2)

In [ ]:
value = 10
driver_share = 0.8
RPH_by_time_75 = hours_36["RPH_by_time"].quantile(0.75)
hours_36['Potential'] = (hours_36.Unavailable * hours_36.Conversion).round(2)
hours_36['Earn'] = hours_36.RPH_by_time * value * driver_share
hours_36['Additional_hours'] = (hours_36.Potential/RPH_by_time_75).round(2)

hours_36.head(37)

In [ ]:
hours_36.describe()

### Conclusion

Questions for which I need answers.
1. What is the supply to demand dynamic and whether they match?
2. Where are the hours of oversupply? Can we shift some of them to undersupply hours?